# Búsqueda en Espacios de Estados
Méndez Pool Joan de Jesús | 160300102

In [1]:
import numpy as np
import sys
from time import time

In [2]:
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [3]:
class Node:  
    # Movimientos permitidos
    moveset = {"up": (-1,0), "down": (1,0), "left":(0,-1), "right":(0,1) }
    #moveset = [ (-1,0), (1,0), (0,-1), (0,1) ]
    # Constructor de la clase
    def __init__ (self, state, father):
        self.state = state
        pos = np.where(self.state==0)
        (y,x) = (pos[0][0], pos[1][0])
        self.position = (y,x)
        self.father = father
    # Verifica si el movimiento es lógicamente posible
    def Range_Grid(self, position):
        bounds = self.state.shape
        cy, cx = position
        flag = np.logical_and([cx >= 0, cy >= 0], [cx < bounds[1], cy < bounds[0]])
        return np.logical_and(flag[0], flag[1])
    
    # Crea una lista de la posibilidad de movimientos disponibles sin retroceder a un estado anterior
    def Childs(self):
        ls = []
        (y,x) = self.position
        for key, (cy, cx) in self.moveset.items():
        #for key, (cy,cx) in self.moveset.items()
            if self.Range_Grid((cy+y,cx+x)):
                arr = self.Swap((cy+y, cx+x))
                if not np.array_equal(arr, self.father):
                    g = Node(np.copy(arr), np.copy(self.state))
                    ls.append(g)
        return ls
    # Intercambia celdas del juego       
    def Swap(self, pos):
        (y,x) = self.position
        (cy, cx) = pos
        arr = np.copy(self.state)
        #Swap
        temp = arr[y][x]
        arr[y][x] = arr[cy][cx]
        arr[cy][cx] = temp
        
        return arr

In [4]:
def reconstruct_path(explored, current):
    # Creamos una lista donde se guardará cada pixel del camino encontrado
    path = [np.array_str(current.state)]
    # Obtenemos el nodo padre de la meta
    api = explored[0][explored[1].index(np.array_str(current.father))]
    # Mientras el nodo padre del nodo actual no apunte a si mismo
    while not np.array_equal(api.state, api.father):
        # Guarda el nodo en la lista de pixeles que representan el camino
        path.append(np.array_str(api.state))
        # Moverse al nodo padre del nodo actual
        api = explored[0][explored[1].index(np.array_str(api.father))]
    # Apilar el nodo de inicio del camino
    path.append(np.array_str(api.state))
    # Regresa la lista invertida, memoria usada por los nodos explorados, número de nodos explorados
    return (path[::-1], get_size(explored), len(explored[0]))

In [5]:
# Breadth First Search
def BFS(Node, Goal):        
    queue = [ [Node], [np.array_str(Node.state)]]
    explored = [ [], []]
    
    while queue[0]:
        s = queue[0].pop(0)
        nll = queue[1].pop(0)
        explored[0].append(s)
        explored[1].append(np.array_str(s.state))
        if np.array_equal(s.state, Goal):
             return reconstruct_path(explored, s)

        for node in s.Childs():
            string = np.array_str(node.state)
            if (not string in queue[1]) or (not string in explored[1]):
                queue[0].append(node)
                queue[1].append(string)
    return False

# Depth First Search
def DFS(Node, Goal):        
    stack = [ [Node], [np.array_str(Node.state)]]
    explored = [ [], []]
    
    while stack[0]:
        s = stack[0].pop()
        nll = stack[1].pop()
        explored[0].append(s)
        explored[1].append(np.array_str(s.state))
        if np.array_equal(s.state, Goal):
             return reconstruct_path(explored, s)

        for node in reversed(s.Childs()):
            string = np.array_str(node.state)
            if (not string in stack[1]) or (not string in explored[1]):
                stack[0].append(node)
                stack[1].append(string)
    return False

# Iterative Deepening Depth First Search
def IDDFS(Node, Goal):
    global explo
    explo[0].append(Node)
    explo[1].append(np.array_str(Node.state))
    for depth in range(7):
        found, remaining = DLS(Node, Goal, depth)
        if not(found==None):
            return reconstruct_path(explo, found)
        elif not remaining:
            return None
        
# Depth Limited Search
explo = [[], []]
def DLS(Node, Goal, depth):
    if depth == 0:
        if np.array_equal(Node.state, Goal):
            return (Node, True)
        else:
            return (None, True)
    elif depth > 0:
        any_remaining = False
        for g in Node.Childs():
            cad = np.array_str(g.state)
            if not cad in explo[1]:
                explo[0].append(g)
                explo[1].append(np.array_str(g.state))
            found, remaining = DLS(g, Goal, depth-1)
            if not(found == None):
                return (found, True)
            if remaining:
                any_remaining = True
        return (None, any_remaining)

In [6]:
def Print_Format(G, Goal, ls, exe_time ):
    print ("Start\n\n" + str(G.state))
    print ("\nGoal\n\n" + str(Goal) + "\n\n#############\n")
    print("Path\n\n")
    for i,g in enumerate(ls[0]):
        print str(i+1) + ".- Step\n" + g + "\n"
    print "Número de Nodos explorados: " + str(ls[2])
    print "Memoria usada por Nodos Explorados: " + str(ls[1]) + " bytes"
    print "El tiempo de ejecución fue: " + str(exe_time) + " mili-segundos"

In [8]:
# goal
goal = np.array([[1,2,3], [4,5,6], [7,8,0]])

# Casos
state1 = np.array([[1,8,4], [3,0,7], [5,6,2]])
state2 = np.array([[1,0,3], [4,2,6], [7,5,8]])
state3 = np.array([[0,1,2], [4,5,3], [7,8,6]])

G1 = Node(np.copy(state1), np.copy(state1))
G2 = Node(np.copy(state2), np.copy(state2))
G3 = Node(np.copy(state3), np.copy(state3))

# Caso 1:

In [ ]:
start_time = time()
ls = BFS(G1, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G1, goal, ls, exe_time)

In [9]:
start_time = time()
ls = DFS(G1, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G1, goal, ls, exe_time)

Start

[[1 2 3]
 [0 5 6]
 [4 7 8]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[1 2 3]
 [0 5 6]
 [4 7 8]]

2.- Step
[[1 2 3]
 [4 5 6]
 [0 7 8]]

3.- Step
[[1 2 3]
 [4 5 6]
 [7 0 8]]

4.- Step
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Número de Nodos explorados: 4
Memoria usada por Nodos Explorados: 4362 bytes
El tiempo de ejecución fue: 0.00963020324707 mili-segundos


In [10]:
start_time = time()
ls = IDDFS(G1, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G1, goal, ls, exe_time)

Start

[[1 2 3]
 [0 5 6]
 [4 7 8]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[1 2 3]
 [0 5 6]
 [4 7 8]]

2.- Step
[[1 2 3]
 [4 5 6]
 [0 7 8]]

3.- Step
[[1 2 3]
 [4 5 6]
 [7 0 8]]

4.- Step
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Número de Nodos explorados: 10
Memoria usada por Nodos Explorados: 10074 bytes
El tiempo de ejecución fue: 0.0181300640106 mili-segundos


# Caso 2:

In [11]:
start_time = time()
ls = BFS(G2, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G2, goal, ls, exe_time)

Start

[[1 0 3]
 [4 2 6]
 [7 5 8]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[1 0 3]
 [4 2 6]
 [7 5 8]]

2.- Step
[[1 2 3]
 [4 0 6]
 [7 5 8]]

3.- Step
[[1 2 3]
 [4 5 6]
 [7 0 8]]

4.- Step
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Número de Nodos explorados: 10
Memoria usada por Nodos Explorados: 10074 bytes
El tiempo de ejecución fue: 0.0174000263214 mili-segundos


In [12]:
start_time = time()
ls = DFS(G2, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G2, goal, ls, exe_time)

Start

[[1 0 3]
 [4 2 6]
 [7 5 8]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[1 0 3]
 [4 2 6]
 [7 5 8]]

2.- Step
[[1 2 3]
 [4 0 6]
 [7 5 8]]

3.- Step
[[1 2 3]
 [4 5 6]
 [7 0 8]]

4.- Step
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Número de Nodos explorados: 4
Memoria usada por Nodos Explorados: 4362 bytes
El tiempo de ejecución fue: 0.011598110199 mili-segundos


In [13]:
start_time = time()
ls = IDDFS(G1, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G2, goal, ls, exe_time)

Start

[[1 0 3]
 [4 2 6]
 [7 5 8]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[1 2 3]
 [0 5 6]
 [4 7 8]]

2.- Step
[[1 2 3]
 [4 5 6]
 [0 7 8]]

3.- Step
[[1 2 3]
 [4 5 6]
 [7 0 8]]

4.- Step
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Número de Nodos explorados: 11
Memoria usada por Nodos Explorados: 10138 bytes
El tiempo de ejecución fue: 0.0298991203308 mili-segundos


# Caso 3:

In [14]:
start_time = time()
ls = BFS(G3, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G3, goal, ls, exe_time)

Start

[[0 1 2]
 [4 5 3]
 [7 8 6]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[0 1 2]
 [4 5 3]
 [7 8 6]]

2.- Step
[[1 0 2]
 [4 5 3]
 [7 8 6]]

3.- Step
[[1 2 0]
 [4 5 3]
 [7 8 6]]

4.- Step
[[1 2 3]
 [4 5 0]
 [7 8 6]]

5.- Step
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Número de Nodos explorados: 30
Memoria usada por Nodos Explorados: 28778 bytes
El tiempo de ejecución fue: 0.109880924225 mili-segundos


In [15]:
start_time = time()
ls = DFS(G3, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G3, goal, ls, exe_time)

Start

[[0 1 2]
 [4 5 3]
 [7 8 6]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[0 1 2]
 [4 5 3]
 [7 8 6]]

2.- Step
[[4 1 2]
 [0 5 3]
 [7 8 6]]

3.- Step
[[4 1 2]
 [7 5 3]
 [0 8 6]]

4.- Step
[[4 1 2]
 [7 5 3]
 [8 0 6]]

5.- Step
[[4 1 2]
 [7 5 3]
 [8 6 0]]

6.- Step
[[4 1 2]
 [7 5 0]
 [8 6 3]]

7.- Step
[[4 1 0]
 [7 5 2]
 [8 6 3]]

8.- Step
[[4 0 1]
 [7 5 2]
 [8 6 3]]

9.- Step
[[4 5 1]
 [7 0 2]
 [8 6 3]]

10.- Step
[[4 5 1]
 [7 6 2]
 [8 0 3]]

11.- Step
[[4 5 1]
 [7 6 2]
 [8 3 0]]

12.- Step
[[4 5 1]
 [7 6 0]
 [8 3 2]]

13.- Step
[[4 5 0]
 [7 6 1]
 [8 3 2]]

14.- Step
[[4 0 5]
 [7 6 1]
 [8 3 2]]

15.- Step
[[4 6 5]
 [7 0 1]
 [8 3 2]]

16.- Step
[[4 6 5]
 [7 3 1]
 [8 0 2]]

17.- Step
[[4 6 5]
 [7 3 1]
 [8 2 0]]

18.- Step
[[4 6 5]
 [7 3 0]
 [8 2 1]]

19.- Step
[[4 6 0]
 [7 3 5]
 [8 2 1]]

20.- Step
[[4 0 6]
 [7 3 5]
 [8 2 1]]

21.- Step
[[4 3 6]
 [7 0 5]
 [8 2 1]]

22.- Step
[[4 3 6]
 [7 2 5]
 [8 0 1]]

23.- Step
[[4 3 6]
 [7 2 5]
 [8 1 0]]

24.- Step
[[4 3 6]


In [16]:
start_time = time()
ls = IDDFS(G3, goal)
end_time = time()
exe_time = end_time - start_time
# Imprimir Resultado
Print_Format(G3, goal, ls, exe_time)

Start

[[0 1 2]
 [4 5 3]
 [7 8 6]]

Goal

[[1 2 3]
 [4 5 6]
 [7 8 0]]

#############

Path


1.- Step
[[0 1 2]
 [4 5 3]
 [7 8 6]]

2.- Step
[[1 0 2]
 [4 5 3]
 [7 8 6]]

3.- Step
[[1 2 0]
 [4 5 3]
 [7 8 6]]

4.- Step
[[1 2 3]
 [4 5 0]
 [7 8 6]]

5.- Step
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Número de Nodos explorados: 40
Memoria usada por Nodos Explorados: 37298 bytes
El tiempo de ejecución fue: 0.13102889061 mili-segundos
